In [2]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import csv
import numpy as np
import os 

from sklearn.feature_extraction.text import CountVectorizer
import nltk, re
from nltk.corpus import stopwords
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model 

import pickle
from sklearn.externals import joblib

In [64]:
# import pickle
# from sklearn.externals import joblib

In [ ]:
###CLASSIFICATION SVM BOUNDING BOX DISTRIBUTION

In [69]:
# loaded = np.load('/tmp/distribution_data.npz')
loaded = np.load('/tmp/ml_classification.npz')
hog_pos_train=loaded['hog_pos_train']
hog_pos_train_label=loaded['hog_pos_train_label']
hog_pos_test=loaded['hog_pos_test'] 
hog_pos_test_label=loaded['hog_pos_test_label']

In [71]:
def correctShape(data):
    num_samples = len(data)
    len_samples = len(data[0])
    new_data = []
    count = 0

    for i in range(num_samples):
        new_data.append([data[i][j][0] for j in range(len_samples)])    

        if count % 5000 == 0:
            print(count)
        count += 1 
        
    return new_data

In [73]:
hog_pos_train = correctShape(hog_pos_train)
hog_pos_test = correctShape(hog_pos_test)

print(len(hog_pos_train[0]))
print(len(hog_pos_test[0]))

print(len(hog_pos_train))
print(len(hog_pos_test))

1568
1568
15000
3971


In [61]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1000, copy=True, whiten=False, svd_solver='auto', tol=0.0, iterated_power='auto', random_state=None)
pca_trainVectors_hog = pca.fit_transform(hog_pos_train)
pca_testVectors_hog = pca.transform(hog_pos_test)
testLabels_hog = hog_pos_test_label
trainLabels_hog = hog_pos_train_label
print(len(pca_trainVectors_hog[0]))
print(len(pca_testVectors_hog[0]))

1000
1000


In [62]:
linear_svc_hog = svm.LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=1.0, multi_class='ovr', 
                    fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=1, random_state=None, 
                    max_iter=1000)
linear_svc_hog.fit(pca_trainVectors_hog, trainLabels_hog)
linear_svc_hog_prediction = linear_svc_hog.predict(pca_testVectors_hog)
print("Accuracy: " + str(metrics.accuracy_score(linear_svc_hog_prediction, testLabels_hog)))
print("F1: " + str(metrics.f1_score(linear_svc_hog_prediction, testLabels_hog, average=None)))

[LibLinear]Accuracy: 0.9579451019894233
F1: [0.90122087 0.90761421 0.97766097 0.83489097 0.97777778 0.98762376
 0.998      0.99797571 1.         0.99373695]


In [65]:
linear_svc_hog.coef_.shape

(10, 1000)

In [66]:
#persist model for later use
joblib.dump(linear_svc_hog, 'svc_sliding_window_classification.pkl') 
#clf = joblib.load('svc_sliding_window_classification.pkl') 

['svc_sliding_window_classification.pkl']

In [ ]:
##DETECTION SVM BOUNDING BOX DISTRIBUTION

In [74]:
##Run SVM Classifier for Dectection on HoG
loaded = np.load('/tmp/ml_detection.npz')
total_hog_train=loaded['total_hog_train'] 
total_hog_train_labels=loaded['total_hog_train_labels']    
total_hog_test=loaded['total_hog_test']
total_hog_test_labels=loaded['total_hog_test_labels']

In [75]:
print(total_hog_train.shape)

(25000, 1568, 1)


In [79]:
total_hog_train = correctShape(total_hog_train)
total_hog_test = correctShape(total_hog_test)
print(len(total_hog_train))
print(len(total_hog_test))

25000
8000


In [81]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1000, copy=True, whiten=False, svd_solver='auto', tol=0.0, iterated_power='auto', random_state=None)
pca_trainVectors_hog = pca.fit_transform(total_hog_train)
pca_testVectors_hog = pca.transform(total_hog_test)
testLabels_hog = total_hog_test_labels
trainLabels_hog = total_hog_train_labels
print(len(pca_trainVectors_hog[0]))
print(len(pca_testVectors_hog[0]))

1000
1000


In [82]:
linear_svc_hog = svm.LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=1.0, multi_class='ovr', 
                    fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=1, random_state=None, 
                    max_iter=1000)
linear_svc_hog.fit(pca_trainVectors_hog, trainLabels_hog)
linear_svc_hog_prediction = linear_svc_hog.predict(pca_testVectors_hog)
print("Accuracy: " + str(metrics.accuracy_score(linear_svc_hog_prediction, testLabels_hog)))
print("F1: " + str(metrics.f1_score(linear_svc_hog_prediction, testLabels_hog, average=None)))

[LibLinear]Accuracy: 0.98875
F1: [0.98819517 0.98925501]


In [92]:
s = pickle.dumps(linear_svc_hog)
np.savez('/tmp/svc_detection_data', classifier=[s], pca_testVectors_hog=pca_testVectors_hog, testLabels_hog=testLabels_hog)

In [83]:
#persist model for later use
joblib.dump(linear_svc_hog, 'svc_sliding_window_detection.pkl') 
#clf = joblib.load('svc_sliding_window_detection.pkl') 

['svc_sliding_window_detection.pkl']